In [6]:
from mpi4py import MPI
import numpy as np
from dolfinx import fem, default_scalar_type, plot, mesh
import ufl
from dolfinx.fem.petsc import LinearProblem
import pyvista  
import helpfunctions 


In [7]:

    
# A unit mesh of 8*8 quadrilateral cells: Dicreticise the domain into finite elements
domain = loadmesh("arolla.xdmf")


# Finite element function space of Langrange polynomials of degree 1
V = fem.functionspace(domain, ("Lagrange", 1))

# Dirichlets BC iterpolated on boundary
uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)


# Mapping cells to facets
tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

# Mapping BC 
boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)

# Defining Trial and Test fucntions 
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# source term over domain
f = fem.Constant(domain, default_scalar_type(-6))

# Writing the problem in variational: Bilinear and Linear
a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

# Defining the problem: Assembly
problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
    petsc_options_prefix="Poisson",
)

#Solving the problem
uh = problem.solve()



In [5]:

#Visualise: Mesh
print(pyvista.global_theme.jupyter_backend)

domain.topology.create_connectivity(tdim, tdim)
topology, cell_types, geometry = plot.vtk_mesh(domain, tdim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    figure = plotter.screenshot("glacier_mesh.png")
plotter.show()
plotter.screenshot("glacier_mesh.png")


#if not pyvista.OFF_SCREEN:
#        plotter.show()
#    else:
#        figure = plotter.screenshot("fundamentals_mesh.png")
        
#Visualize: the solution, DOF in 2D
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(V)

u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=True)
u_plotter.view_xy()
if not pyvista.OFF_SCREEN:
    u_plotter.show()
#Visualise:  the solution in 3D, Warped
warped = u_grid.warp_by_scalar()
plotter2 = pyvista.Plotter()
plotter2.add_mesh(warped, show_edges=True, show_scalar_bar=True)
if not pyvista.OFF_SCREEN:
    plotter2.show()



: 

In [ ]:
#Calculating the error:Interpolate exact solution in function space
V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2, name="u_exact")
uex.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)

#L2 norm error: 
L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

#Maximum error at any DOF:
error_max = numpy.max(numpy.abs(uD.x.array - uh.x.array))

# Only print the error on one process
if domain.comm.rank == 0:  
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")
